<a href="https://colab.research.google.com/github/passion1014/rag/blob/main/LCEL_Test_Chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Common Setting

In [29]:
# !pip install langchain
# !pip install langchain_core
# !pip install langchain_openai
# !pip install langchain_community
# !pip install langchain_experimental
# !pip install qrcode
# !pip install geocoder
# !pip install fpdf
# !pip install requests
# !pip install python-pptx
!pip install pyqrcode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyqrcode: filename=PyQRCode-1.2.1-py3-none-any.whl size=36223 sha256=265d3958ba3eb892b6da618e81eb0fe67370dedb6db7e3ee9f239eae7dad779c
  Stored in directory: /root/.cache/pip/wheels/34/2d/20/082874e49a32cdcc87ebaf99dd0099fb7774ef30f010dfb6f2
Successfully built pyqrcode


In [2]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [26]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import numexpr
from langchain_core.output_parsers import StrOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain_core.runnables import RunnablePassthrough

# Math Chain

In [9]:
prompt_template = ChatPromptTemplate.from_template(
    "For {query}, write only the mathematical expression suitable for numexpr.evaluate().")
chain = prompt_template | ChatOpenAI() | StrOutputParser() | numexpr.evaluate
chain.invoke({'query' : 'What is 13 raised to the .3432 power?'})

array(2.41160046)

In [7]:
numexpr.evaluate('13**.3432')

array(2.41160046)

# SQL Chain

In [11]:
import urllib.request

urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sql",
    filename="Chinook_Sqlite.sql",
)

('Chinook_Sqlite.sql', <http.client.HTTPMessage at 0x798587ea7700>)

In [13]:
import sqlite3

conn = sqlite3.connect('Chinook.db')

with open("Chinook_Sqlite.sql", 'r') as file:
    script = file.read()

conn.executescript(script)
conn.close()

In [14]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [19]:
prompt_template = ChatPromptTemplate.from_template("""Based on the table schema below,
write a SQL query that would answer the user's question:

{schema}

Question: {question}
SQL Query:""")

def get_schema(_):
    return db.get_table_info()

chain = (RunnablePassthrough.assign(schema = get_schema)
         | prompt_template
         | ChatOpenAI()
         | StrOutputParser()
         | db.run)

In [20]:
chain.invoke({"question":'점원의 수는?'})

'[(8,)]'

# Code Chain

In [31]:
template = """Write some python code to solve the user's problem.
Do not ask the user again for confirmation or additional information.
Return must only python code that can be directly executed."""

prompt_template = ChatPromptTemplate.from_messages(
    [("system", template),
     ("human", "{user_input}")]
)

chain = {"user_input": RunnablePassthrough()} | prompt_template | ChatOpenAI() | StrOutputParser() | PythonREPL().run
chain.invoke("https://tykimos.github.io QR코드 이미지 생성해줘")

''